# AMP-SOA portfolio optimization using adaptive meta-policy based on second-order agents with deep reinforcment learning
---

## 6.0 Data Split
---

We will split both the close prices and the whole dataset into train and test (trade) data.

We will use 80% of the data for training and then test on the remaining 20%.

We will make use of the FinRL Library function of data_split to split our data into train and test

### 6.1 Import Relevant Libraries

In [1]:
import pandas as pd
import numpy as np
import ta
from ta import add_all_ta_features
from ta.utils import dropna
from finrl.preprocessing.data import data_split
from finrl.preprocessing.preprocessors import FeatureEngineer

In [2]:
df_close_full_stocks = pd.read_csv('datasets/close_prices.csv')

### 6.2 Load the data

In [3]:
filtered_stocks = pd.read_csv('filtered_stocks.csv')
# df_close_full_stocks = pd.read_csv('datasets/close_prices.csv')
data_df = pd.read_csv('data_df.csv')
filtered_stocks = filtered_stocks.drop(columns=['Unnamed: 0'])
filtered_stocks = filtered_stocks['stock_name'].tolist()
data_df = data_df.drop(columns=['Unnamed: 0'])

#%store filtered_stocks
#%store df_close_full_stocks
#%store data_df

%store -r filtered_stocks
# %store -r df_close_full_stocks
%store -r data_df

In [4]:
data_df.head()

,date,tic,close,high,low,open,volume,cov_list,f01,f02,f03,f04
0,2009-03-20,AAPL,3.067483,3.6825,3.591786,3.646071,695587200.0,"[[0.0013155151011021062, 0.0011575384750591776...",1.766427,4.765831,0.804321,1.373377
1,2009-03-20,AXP,9.629786,13.1900,12.120000,13.190000,31088200.0,"[[0.0013155151011021062, 0.0011575384750591776...",1.766427,4.765831,0.804321,1.373377
2,2009-03-20,CAT,17.987209,28.9000,26.730000,28.629999,16531300.0,"[[0.0013155151011021062, 0.0011575384750591776...",1.766427,4.765831,0.804321,1.373377
3,2009-03-20,CSCO,10.697708,16.5700,15.750000,16.370001,66078200.0,"[[0.0013155151011021062, 0.0011575384750591776...",1.766427,4.765831,0.804321,1.373377
4,2009-03-20,HD,15.282681,22.7300,21.760000,22.590000,22361800.0,"[[0.0013155151011021062, 0.0011575384750591776...",1.766427,4.765831,0.804321,1.373377


In [5]:
df_close_full_stocks.head()

,date,CAT,MMM,PG,BA,VZ,WMT,GS,DD,INTC,...,IBM,KO,JNJ,XOM,AXP,HD,RTX,V,DIS,MSFT
0,2008-03-19,47.481430,40.100849,42.133179,54.094528,14.193682,11.901863,127.170723,32.205429,13.014485,...,64.913170,18.092833,40.176636,46.908852,32.275002,17.789188,29.209347,12.605216,26.296064,20.830700
1,2008-03-20,47.539413,39.348988,43.102489,55.088776,14.593674,12.476101,137.207535,32.267361,13.421768,...,65.684784,18.418718,40.486256,47.225517,35.333435,18.597795,29.315559,14.356564,26.851618,21.238287
2,2008-03-24,48.962597,39.682026,43.475292,56.186153,14.937099,12.569845,136.634583,33.311092,13.656260,...,66.089989,18.460970,40.182823,47.753345,36.432312,19.393135,29.880531,13.325836,26.969458,21.231007
3,2008-03-25,49.355450,39.853596,43.208122,55.898914,14.908816,12.433908,137.207535,33.487988,13.742653,...,65.484924,18.530369,39.972290,47.336658,36.509171,19.061741,29.804073,14.111153,27.003124,21.209167
4,2008-03-26,49.819111,39.692108,43.270245,56.193516,14.601750,12.398750,134.037613,33.443768,13.489645,...,64.896507,18.451914,40.065163,47.925556,34.857006,18.664076,29.570442,14.269552,26.733772,20.787031


In [6]:
# Close Prices data frame

# Reset the Index to tic and date
df_prices = data_df.reset_index().set_index(['tic', 'date']).sort_index()

# Get all the Close Prices
df_close = pd.DataFrame()

for ticker in filtered_stocks:
    series = df_prices.xs(ticker).close
    df_close[ticker] = series

In [7]:
df_close.head()

,PG,JNJ,VZ,KO,PFE,MCD,MMM,IBM,WMT,RTX,HD,MRK,CSCO,CAT,V,AAPL,AXP,MSFT,XOM,TRV
date,,,,,,,,,,,,,,,,,,,,
2009-03-20,29.053087,32.971901,12.690037,13.293597,7.213285,34.688774,23.939526,52.335415,11.836153,17.653542,15.282681,15.113528,10.697708,17.987209,11.782366,3.067483,9.629786,12.683455,37.472717,26.869574
2009-03-23,30.423212,33.967369,13.315755,13.761237,7.419681,35.966774,25.706448,55.842934,12.287255,18.938471,16.034403,15.826218,11.417165,19.688246,12.457502,3.250766,11.436350,13.627644,39.990181,28.323477
2009-03-24,29.996241,33.629177,13.075748,13.723833,7.366760,34.923508,25.174807,55.610977,12.191789,18.820862,15.827507,15.696126,11.215447,19.535423,12.022358,3.215739,10.917939,13.330261,39.338123,27.146507
2009-03-25,30.627136,33.731262,12.921464,13.951415,7.546693,35.868965,25.550093,55.412983,12.334992,19.029938,16.055084,15.554715,11.141482,19.209833,11.831709,3.215437,11.082889,13.293095,39.780380,27.845755
2009-03-26,31.073227,33.756805,13.088610,13.982594,7.610200,36.553623,26.597729,55.882519,12.592767,19.805252,16.648190,15.599963,11.639051,20.452387,12.448528,3.317496,11.884064,13.999380,40.387074,27.734989


In [8]:
df_close = df_close.reset_index()

### 6.3 Split the Data

In [9]:
# Define the start and end dates for the train and test data

train_pct = 0.8 # percentage of train data

date_list = list(data_df.date.unique()) # List of dates in the data

date_list_len = len(date_list) # len of the date list
train_data_len = int(train_pct * date_list_len) # length of the train data

train_start_date = date_list[0]
train_end_date = date_list[train_data_len]

test_start_date = date_list[train_data_len+1]
test_end_date = date_list[-1]

In [10]:
print('Training Data: ', 'from ', train_start_date, ' to ', train_end_date)

Training Data:  from  2009-03-20  to  2021-03-26


In [11]:
print('Testing Data: ', 'from ', test_start_date, ' to ', test_end_date)

Testing Data:  from  2021-03-29  to  2024-03-28


In [12]:
df_close_full_stocks

,date,CAT,MMM,PG,BA,VZ,WMT,GS,DD,INTC,...,IBM,KO,JNJ,XOM,AXP,HD,RTX,V,DIS,MSFT
0,2008-03-19,47.481430,40.100849,42.133179,54.094528,14.193682,11.901863,127.170723,32.205429,13.014485,...,64.913170,18.092833,40.176636,46.908852,32.275002,17.789188,29.209347,12.605216,26.296064,20.830700
1,2008-03-20,47.539413,39.348988,43.102489,55.088776,14.593674,12.476101,137.207535,32.267361,13.421768,...,65.684784,18.418718,40.486256,47.225517,35.333435,18.597795,29.315559,14.356564,26.851618,21.238287
2,2008-03-24,48.962597,39.682026,43.475292,56.186153,14.937099,12.569845,136.634583,33.311092,13.656260,...,66.089989,18.460970,40.182823,47.753345,36.432312,19.393135,29.880531,13.325836,26.969458,21.231007
3,2008-03-25,49.355450,39.853596,43.208122,55.898914,14.908816,12.433908,137.207535,33.487988,13.742653,...,65.484924,18.530369,39.972290,47.336658,36.509171,19.061741,29.804073,14.111153,27.003124,21.209167
4,2008-03-26,49.819111,39.692108,43.270245,56.193516,14.601750,12.398750,134.037613,33.443768,13.489645,...,64.896507,18.451914,40.065163,47.925556,34.857006,18.664076,29.570442,14.269552,26.733772,20.787031
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4030,2024-03-22,356.809326,88.665146,160.622849,188.850006,39.087002,60.660030,404.372894,75.388237,42.397793,...,188.964172,60.024254,153.985260,112.575607,224.593124,387.585602,94.968666,282.736755,115.337891,427.968048
4031,2024-03-25,354.657196,87.054260,159.162277,191.410004,39.571114,60.361069,402.504181,75.587311,41.660786,...,186.934326,59.934944,153.975357,113.726265,224.662674,380.862335,95.058121,280.690521,118.811867,422.098633
4032,2024-03-26,355.095612,85.219177,159.519974,187.500000,39.551746,60.301273,403.736725,75.189171,41.820141,...,186.647186,60.073868,154.520950,112.873192,223.102173,377.307037,95.485550,280.081665,119.379250,420.890808
4033,2024-03-27,363.325592,86.846680,161.566742,191.949997,40.219822,60.510551,412.752167,76.144707,43.592937,...,188.924576,60.560093,156.693390,114.043686,226.372269,383.225922,96.867241,278.504578,120.424423,420.671204


In [13]:
# Split the whole dataset
train_data = data_split(data_df, train_start_date, train_end_date)
test_data = data_split(data_df, test_start_date, test_end_date)

# Split the Close Prices dataset
prices_train_data = df_close[df_close['date']<=train_end_date]
prices_test_data = df_close[df_close['date']>=test_start_date]

# split the Close Prices of all stocks
prices_full_train = df_close_full_stocks[df_close_full_stocks['date']<=train_end_date]
prices_full_test = df_close_full_stocks[df_close_full_stocks['date']>=test_start_date]

### 6.4 Store the Dataframes

In [14]:
prices_train = prices_train_data.copy()
prices_test = prices_test_data.copy()

train_df = train_data.copy()
test_df = test_data.copy()

prices_full_train_df = prices_full_train.copy()
prices_full_test_df = prices_full_test.copy()

In [15]:
prices_train.to_csv('./prices_train.csv')
prices_test.to_csv('./prices_test.csv')

train_df.to_csv('./train_df.csv')
test_df.to_csv('./test_df.csv')

prices_full_train_df.to_csv('./prices_full_train_df.csv')
prices_full_train_df.to_csv('./prices_full_test_df.csv')

%store prices_train
%store prices_test

%store train_df
%store test_df

%store prices_full_train_df
%store prices_full_test_df

Stored 'prices_train' (DataFrame)
Stored 'prices_test' (DataFrame)
Stored 'train_df' (DataFrame)
Stored 'test_df' (DataFrame)
Stored 'prices_full_train_df' (DataFrame)
Stored 'prices_full_test_df' (DataFrame)
